# Lesson 4: Pre-processing Part 2 - Rank Filters

Here we continue from Pre-processing Part 1, and introduce the following concepts
- The limits of Otsu's method to find global thresholds
- The difference between global and local processing methods
- A few useful local pre-processing operations
    - Using the median filter to remove noise
    - Using rolling ball background subtraction
- The effect of varying the structuring element on rank filters
- How to obtain useful global metrics from processed images

First, let's reestablish the environment we set up in the previous lesson's notebook.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
sns.set_style('dark', rc={'image.cmap':'inferno'})
import matplotlib.axes as ax

data_nodrug = imread("../data/confocal_drug_panel/DMSO.tif")

import json
with open('../data/confocal_drug_panel/DMSO_metadata.json', mode='r') as f_nodrug:
    meta_nodrug = json.load(f_nodrug)

nodrug_slice = {}
for idx, channel in enumerate(meta_nodrug['channels']):
    nodrug_slice[channel] = data_nodrug[2,:,:,idx]    
data = nodrug_slice['actin']

Visualize the raw data and masks generate from global Otsu's threshold in Pre-processing Part 1 - Global Thresholds

In [ ]:
#parameters to adjust
minX1 = 450 #crop edges for a cell in the center of field of view
minY1 = 250
minX2 = 0 #crop edges for cell at the edge of the field of view
minY2 = 250
crop_size = 200 #pix
image_view_thresh = 0.7 #set for the entire notebook so can compare outputs more easily

#run
maxX1 = minX1 + crop_size
maxY1 = minY1 + crop_size
maxX2 = minX2 + crop_size
maxY2 = minY2 + crop_size

top = data.max() * image_view_thresh

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[1].imshow(data[minY1 : maxY1 , minX1 : maxX1], vmin=0, vmax=top, interpolation = 'nearest')
ax[0].imshow(data, vmin=0, vmax=top, interpolation = 'nearest')
ax[2].imshow(data[minY2 : maxY2, minX2: maxX2], vmin=0, vmax=top, interpolation = 'nearest')

from skimage import filters
thresh = filters.threshold_otsu(data)
print("the Otsu masking threshold for this dataset is:", thresh)
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
mask = np.zeros(data.shape)
mask[data >=thresh] = 1
mask_zoom_center = mask[minY1 : maxY1 , minX1 : maxX1]
mask_zoom_edge = mask[minY2 : maxY2 , minX2 : maxX2]
ax[0].imshow(mask, vmin=0, vmax=1)
ax[1].imshow(mask_zoom_center, vmin=0, vmax=1)
ax[2].imshow(mask_zoom_edge, vmin=0, vmax=1)

crop_edge = data[minY2 : maxY2, minX2: maxX2]
crop_mid = data[minY1 : maxY1 , minX1 : maxX1]

## Introduction to rank filters: denoising with the median filter

Note the problems with the dataset that produce low-quality ROIs:
1) Noise
2) Uneven illumination

Rank filters are a subset of common image processing tools that modify images pixel-by-pixel by including information about the surrounding pixels. They do the heavy lifting in many algorithms for denoising (here the median filter), flattening illumination or background, and morphological manipulations. 

The skimage piece on rank filters (http://scikit-image.org/docs/dev/auto_examples/xx_applications/plot_rank_filters.html#sphx-glr-auto-examples-xx-applications-plot-rank-filters-py) explains different types of filters and has sample code.

In [ ]:
from skimage.filters.rank import minimum as min_filter
from skimage.morphology import disk
import matplotlib.patches as patches

Because these filters work pixel-by-pixel, let's first zoom in on a small portion of our original dataset. Note that we specify 'nearest' interpolation so that imshow does not blur our image.

In [ ]:
zoom = crop_edge[75:95 , 50:70]
plt.imshow(zoom, interpolation='nearest', vmin = 0, vmax = top)

Let's simulate an abberantly dark pixel at a known location (to protect your lesson from variation in input datasets) and an abberantly bright pixel.

In [ ]:
dark_pix_x = 10
dark_pix_y = 5
bright_pix_x = 15
bright_pix_y = 5

zoom_noised = zoom.copy()
zoom_noised[bright_pix_y, bright_pix_x] = zoom.max()*2
zoom_noised[dark_pix_y, dark_pix_x] = 0
plt.imshow(zoom_noised, interpolation='nearest', vmin = 0, vmax = top)

These abberant pixel values do not accurately reflect the local concentration of your fluorescent protein, but instead a faulty detector on your camera (for ccds or scmos) or noise. Rank filters use the information from the pixels in the neighborhood of this pixel to reassign a value in the "filtered" image.

Let's take a look at the dark pixel in the image and the pixels immediately surrounding it. 

In [ ]:
plt.imshow(zoom_noised, interpolation='nearest')
nhood = 3 #works best with an odd number, but an even number could demonstrate the problem with some even-numbered structuring element sizes

#plt.gca().add_patch(plt.Rectangle((bright_pix_x-nhood/2, bright_pix_y-nhood/2), nhood, nhood, fill=None, color='y', lw=2))
plt.gca().add_patch(plt.Rectangle((dark_pix_x-nhood/2, dark_pix_y-nhood/2), nhood, nhood, fill=None, color='y', lw=2))

In [ ]:
print("There is a lot of information about what that pixel value could be based on its neighbors. ",
      "Their values are:")
NHOOD = zoom_noised[int(dark_pix_y - (nhood-1)/2 ) : int(dark_pix_y + (nhood-1)/2) +1 , int(dark_pix_x - (nhood-1)/2) : int(dark_pix_x + (nhood-1)/2 )+1]
print( NHOOD)

print("ranked, the values are" , sorted(NHOOD.flatten()) )
print("with a median value of ", np.median(NHOOD))

print("If this were one step in a median filter, the median value in the neighborhood would become", np.median(NHOOD), "in the new, filtered image")

Of course, performing these manipulations on choice pixels is not a reproducible approach, and fortunately there are good built-in 2D median filters that will process each pixel in the image by considering its neighbors. A median filter applied to each pixel in the image above results in noise reduction. 

Apply a median filter with the parameters in the example above using the skimage rank filters and skimage morphology libraries:

In [ ]:
from skimage.filters.rank import median as median_filter
from skimage.morphology import square

plt.imshow(median_filter(zoom_noised, square(nhood) ) , interpolation='nearest', vmin = 0, vmax = top)

This filter was applied to the entire image (pixel by pixel) and resulted in loss of the abberant bright and dark pixels. What else do you notice about the image?

## Structuring elements: determining the neighborhood for rank filters

In the example above, the neighborhood considered for determining the output for each pixel was the set of pixels immediately adjacent to our pixel of interest in a square (the pixels boxed in the example above). The shape and size of the neighborhood used in the algorithm is called the structuring element.

You have total freedom to choose any structuring element you want, but generally simple symmetric shapes are used because their effects are intuitive. In fact, we can let `scikit-image` generate reasonable structuring elements for us! This is a good idea to maximize repeatability.

See: http://scikit-image.org/docs/dev/api/skimage.morphology.html for some options.

"Disk," which approximates a circle around the filtered pixel, is a common choice for structuring element. The skimage.morphology library structuring elements can be viewed directly.

### Rank Filters Example 1: Median Filter

#### Influence of structuring element size on image output

View the size and shape of the "disk" structuring element included in the skimage morpholoogy package. 

In [ ]:
from skimage.morphology import disk

with sns.axes_style('white'):
    N = 5
    fig, axes = plt.subplots(1, N, figsize=(16, 3))
    for n, ax in enumerate(axes):
        np1 = n + 1
        ax.imshow(np.pad(disk(np1), N-n, 'constant'), interpolation='nearest')
        c = plt.Circle((np1 + N - n, np1 + N - n), radius=np1, fill=False, lw=4, color='b')
        ax.add_artist(c)
        ax.set_xlim(0, 2 * N + 2)
        ax.set_ylim(0, 2 * N + 2)

Visualize the effect of changing the size of the structuring element used to median filter the image.

In [ ]:
extreme = 1000

from skimage.filters.rank import median as median_filter
from ipywidgets import interactive

im_filter = zoom_noised

@interactive
def apply_filter(s=(1, 10)):
    # Here we implement the median filtering
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))
    ax[0].imshow(im_filter, interpolation = 'nearest')
    ax[0].set_title('original')
    filtered = median_filter(im_filter, disk(s))
    dif_img = filtered.astype('int') - im_filter.astype('int')
    print("total difference in image =" + str(np.mean(dif_img)) + " arbitrary units")
    print("percent change =" + str(np.mean(dif_img)/100) + "%")                     
    ax[1].imshow(filtered, interpolation = 'nearest')
    ax[1].set_title('filtered with s = {0:d}'.format(s))
    ax[2].imshow(dif_img, vmin=-extreme, vmax=extreme, cmap='coolwarm',interpolation = 'nearest')
    ax[2].set_title('difference between images')
apply_filter

Note that the size of the filter determines the value of the pixels in the output as well as how much the output is blurred by the filter. That means, the larger the filter size, the more neighbours the filter will include before deciding what the new pixel value should be. A good rule of thumb when determining an appropriate filter size is that it should be the smallest filter that sufficiently flattens the visible noise in the background. Many of these operations do not have well-accepted statistical tests for determing the appropriate parameters, so care needs to be taken to record and reproduce processing steps with the same parameters. 

Let's choose a filter size of 2x2.

In [ ]:
f_size = 2
filtered_im = median_filter(data, disk(f_size))
filtered_crop_edge = median_filter(crop_edge, disk(f_size))
filtered_crop_mid = median_filter(crop_mid, disk(f_size))

fig, ax = plt.subplots(2, 3, figsize=(20, 10))
ax[0, 0].imshow(data, interpolation = 'nearest', vmin = 0, vmax = top)
ax[0, 1].imshow(crop_edge, interpolation = 'nearest', vmin = 0, vmax = top)
ax[0, 2].imshow(crop_mid, interpolation = 'nearest', vmin = 0, vmax = top)
ax[1, 0].imshow(filtered_im, interpolation = 'nearest', vmin = 0, vmax = top)
ax[1, 1].imshow(filtered_crop_edge, interpolation = 'nearest', vmin = 0, vmax = top)
ax[1, 2].imshow(filtered_crop_mid, interpolation = 'nearest', vmin = 0, vmax = top)

ax[0, 0].set_title('Original')
ax[0, 1].set_title('Zoom on edge')
ax[0, 2].set_title('Zoom in center')
ax[1, 0].set_title('Filtered')
ax[1, 1].set_title('Filtered: Zoom on edge')
ax[1, 2].set_title('Filetered: Zoom in center')

The median filter removed much of the noise!

Now let's see how the filtering affects our mask, and compare to the mask we made earlier.

**Exercise** Match the output image to the operation that produced it (from a single input image)

Match the sample figure to the type of operation performed on it (with a disk structuring element):

    1) Mean filter, s=10
    2) Min filter, s=10
    3) Max filter, s=10
    4) Mean filter, s=2
    5) Min filter, s=2
    6) Max filter, s=2

In [ ]:
from IPython.display import Image # Access IPython's browser-based image display.
Image("../fig/raw_for_types_of_filters_exercise.png") # Quickly display a diagram we saved in a file

In [ ]:
Image("../fig/types_of_filters_exercise.png") # Quickly display a diagram we saved in a file

In [ ]:
# read image
img = imread("../fig/raw_for_types_of_filters_exercise.png")

# image preprocessing
from skimage.filters.rank import mean as mean_filter
from skimage.filters.rank import minimum as min_filter
from skimage.filters.rank import maximum as max_filter

fig, ax = plt.subplots(2, 3, figsize=(10, 5))
ax[0, 0].imshow(mean_filter(img, disk(10)), interpolation = 'nearest', vmin = 0, vmax = top)
ax[0, 1].imshow(min_filter(img, disk(10)), interpolation = 'nearest', vmin = 0, vmax = top)
ax[0, 2].imshow(max_filter(img, disk(10)), interpolation = 'nearest', vmin = 0, vmax = top)
ax[1, 0].imshow(mean_filter(img, disk(2)), interpolation = 'nearest', vmin = 0, vmax = top)
ax[1, 1].imshow(min_filter(img, disk(2)), interpolation = 'nearest', vmin = 0, vmax = top)
ax[1, 2].imshow(max_filter(img, disk(2)), interpolation = 'nearest', vmin = 0, vmax = top)

ax[0, 0].set_title('Mean fitler, s = 10')
ax[0, 1].set_title('Min fitler, s = 10')
ax[0, 2].set_title('Max fitler, s = 10')
ax[1, 0].set_title('Mean fitler, s = 2')
ax[1, 1].set_title('Min fitler, s = 2')
ax[1, 2].set_title('Max fitler, s = 2')

ax[0, 0].set_xticklabels('')
ax[0, 1].set_xticklabels('')
ax[0, 2].set_xticklabels('')

Back to our actin image, let's now create a mask using the filtered image

In [ ]:
thresh = filters.threshold_otsu(filtered_im)

masked_filtered_im = np.zeros(filtered_im.shape)
masked_filtered_crop_edge = np.zeros(filtered_crop_edge.shape)
masked_filtered_crop_mid = np.zeros(filtered_crop_mid.shape)

masked_filtered_im[filtered_im > thresh] = 1
masked_filtered_crop_edge[filtered_crop_edge > thresh] = 1
masked_filtered_crop_mid[filtered_crop_mid > thresh] = 1

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(mask, vmin=0, vmax=1)
ax[0].set_title('Before filtering')
ax[1].imshow(mask_zoom_center, vmin=0, vmax=1)
ax[2].imshow(mask_zoom_edge, vmin=0, vmax=1)

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(masked_filtered_im, vmin=0, vmax=1)
ax[0].set_title('After filtering')
ax[2].imshow(masked_filtered_crop_edge, vmin=0, vmax=1)
ax[1].imshow(masked_filtered_crop_mid, vmin=0, vmax=1)

In this case, denoising creates smoother masks but the resultant global threshold from Otsu's method still does not handle cells in the center of the image as well as cells at the edge of the image. To understand why, see the histogram. 

In [ ]:
sns.distplot(filtered_im.flatten(), hist_kws={'log': False}, kde=False)
plt.axvline(thresh, ls='--', lw=2, c='r', label='Otsu threshold')
plt.gca().set_ylim([0, 200000])
plt.legend()

### Rank Filters Example 2: Rolling Ball Background Subtraction

The problem in the image comes from non-uniform illumination. Let's now find out the background of the image using a rolling ball or minimum filter.

In [ ]:
structuring_element = disk(101)
background = min_filter(filtered_im, structuring_element)
plt.imshow(background, cmap='inferno', vmin = 0, vmax = top*0.2)

Let's substract this background from the image.

In [ ]:
bgs = filtered_im - background
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(bgs, interpolation = 'nearest', vmin = 0, vmax = top)
ax[1].imshow(bgs[minY1:maxY1, minX1:maxX1], interpolation = 'nearest', vmin = 0, vmax = top*0.5)
ax[2].imshow(bgs[minY2:maxY2, minX2:maxX2], interpolation = 'nearest', vmin = 0, vmax = top*0.5)

Then we can use Otsu's method to find out the global threshold for the background subtracted image.

In [ ]:
thresh = filters.threshold_otsu(bgs)

masked_bgs = np.zeros(bgs.shape)
masked_bgs_crop_edge = np.zeros(bgs[minY1:maxY1, minX1:maxX1].shape)
masked_bgs_crop_mid = np.zeros(bgs[minY2:maxY2, minX2:maxX2].shape)

masked_bgs[bgs > thresh] = 1
masked_bgs_crop_edge[bgs[minY1:maxY1, minX1:maxX1] > thresh] = 1
masked_bgs_crop_mid[bgs[minY2:maxY2, minX2:maxX2] > thresh] = 1

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(masked_filtered_im, vmin=0, vmax=1)
ax[0].set_title('Before background subtraction')
ax[2].imshow(masked_filtered_crop_edge, vmin=0, vmax=1)
ax[1].imshow(masked_filtered_crop_mid, vmin=0, vmax=1)

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(masked_bgs, vmin=0, vmax=1)
ax[0].set_title('After background subtraction')
ax[1].imshow(masked_bgs_crop_edge, vmin=0, vmax=1)
ax[2].imshow(masked_bgs_crop_mid, vmin=0, vmax=1)

Let's see whether Otsu's threshold makes sense for our background subtracted image.

In [ ]:
sns.distplot(bgs.flatten(), hist_kws={'log': False}, kde=False)
plt.gca().set_ylim([0, 40000])
plt.axvline(thresh, ls='--', lw=2, c='r', label='Otsu threshold')
plt.legend()

Note that subtraction of background now separates the intensities into two distinct populations but Otsu's fails because of the shape of the distribution (the foreground has a long tail)

**Exercise** Apply other thresholding methods

Otsu's method performs sub-optimally in this case because of the distribution of background and foreground values. Find the documentation for the scipy filter options and determine if another thresholding algorithm would be more appropriate.

Here is the link: http://scikit-image.org/docs/dev/api/skimage.filters.html

It would be cool if they found the following function in the documentation: 
skimage.filters.try_all_threshold(image, figsize=(8, 5), verbose=True)

(this is listed right below the other filter options)

**CAUTION** thresh_minimum will take forever to run for this particular image, so that it's better not to run try_all_threshold here

In [ ]:
thresh_otsu = filters.threshold_otsu(bgs)
thresh_li = filters.threshold_li(bgs)
thresh_yen = filters.threshold_yen(bgs)

sns.distplot(bgs.flatten(), hist_kws={'log': False}, kde=False)
plt.gca().set_ylim([0, 20000])
plt.gca().set_xlim([0, 20000])
plt.axvline(thresh_otsu, ls='--', lw=2, c='r', label='otsu')
plt.axvline(thresh_li, ls='--', lw=2, c='b', label='li')
plt.axvline(thresh_yen, ls='--', lw=2, c='g', label='yen')
plt.legend()

print('thresh_otsu = ', thresh_otsu)
print('thresh_li = ', thresh_li)
print('thresh_yen = ', thresh_yen)

The Yen's method does a good job in separating the background from the foreground. Now let's apply Yen's threshold to create a new mask.

In [ ]:
thresh = filters.threshold_yen(bgs)

masked_bgs = np.zeros(bgs.shape)
masked_bgs_crop_edge = np.zeros(bgs[minY1:maxY1, minX1:maxX1].shape)
masked_bgs_crop_mid = np.zeros(bgs[minY2:maxY2, minX2:maxX2].shape)

masked_bgs[bgs > thresh] = 1
masked_bgs_crop_edge[bgs[minY1:maxY1, minX1:maxX1] > thresh] = 1
masked_bgs_crop_mid[bgs[minY2:maxY2, minX2:maxX2] > thresh] = 1

top = data.max() * image_view_thresh

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[1].imshow(data[minY1 : maxY1 , minX1 : maxX1], interpolation = 'nearest', vmin = 0, vmax = top)
ax[0].imshow(data, interpolation = 'nearest', vmin = 0, vmax = top)
ax[2].imshow(data[minY2 : maxY2, minX2: maxX2], interpolation = 'nearest', vmin = 0, vmax = top)

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(masked_bgs, vmin=0, vmax=1)
ax[1].imshow(masked_bgs_crop_edge, vmin=0, vmax=1)
ax[2].imshow(masked_bgs_crop_mid, vmin=0, vmax=1)

Next: Morphological Operations. This follows directly from this lesson in that morphological image processing is an implementation of a rank filter. It expands on the idea of different types of structuring elements (kernals) and how they modify images. Cleaning up the masks above with morphological image processing would be necessary to do analysis on whole cells (probably good to remove the small blobs) so can assign pixels to cells. 